# Time Series Analysis and Forecast on Perth Fuel Price (2022-2024)

&emsp;The project follows `Box-Jenkins`'s modelling framework that consists of `identification`, `estimation`, `model diagnostic`, and `production` to practise analysing and making forecast of a time series dataset, Perth's fuel prices. Note that the fuel price is expressed as cents per litre (AUD/100L).

## Table of Contents
1. [Import Library](#lib)
2. [Import Data](#data)  
&emsp;a. [SQL](#sql1)  
&emsp;b. [Pandas](#read_csv)
3. [Data Preparation](#prep)
4. [Modelling](#model)
5. [Exogenous Feature](#exog)

## Import Library<a id='lib'></a>

In [1]:
# MySQL
import psycopg2
import mysql.connector
# Numpy and Dataframe
import numpy as np
import pandas as pd
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.style.use('default')
%matplotlib inline
# Stationarity test
from statsmodels.tsa.stattools import adfuller
# Order searching
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
# Seasonal decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
# Modelling
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Save and load model
import joblib

## Import Data<a id='data'></a>
&emsp;Fuel price data was retrieved from [Fuel Watch](https://www.fuelwatch.wa.gov.au). Multiple Excel files were imported into MySQL database. Only unleaded petrol price in Perth Metro was extracted from the large amount of data for this particular analysis and forecast.

### SQL<a id='sql1'></a>

In [2]:
# Connect database
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   password="password"
# )
# mycursor = mydb.cursor()

In [3]:
# Extract dataset
# mycursor.execute(
#     "SELECT date, brand, price\
#     FROM base.WA_fuel\
#     WHERE (fuel = 'ULP') AND (postcode = 6000)"
# )

# result = mycursor.fetchall()

In [4]:
# for x in result:
#     print(x)

In [5]:
# Turn SQL result into dataframe
# df = pd.DataFrame(result, columns=['date','brand','price'])
# print(f"Number of rows: {df.shape[0]}")
# print(f"Number of columns: {df.shape[1]}")
# df.head(10)

### Pandas<a id='read_csv'></a>

In [6]:
df = pd.read_excel('perth_fuel.xlsx').iloc[:,1:]

In [ ]:
df.drop_duplicates(inplace=True)
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
df.reset_index(drop=True, inplace=True)
df.head(10)

In [8]:
# Convert date column into datetime
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [ ]:
df.head()

## Data Preparation<a id='prep'></a>

In [10]:
# Congregate data using months
df['month_year'] = df['date'].dt.to_period('M')
df['date'] = df['month_year']

In [11]:
# Group dataset by date and aggregate by price's mean
df_uni = df.groupby(['date'])['price'].agg('mean')
# Narrow down to 3 years data
df_uni = df_uni[36:]

### Stationarity

In [ ]:
# Testing trend stationarity
fig, ax = plt.subplots(figsize=(8,5))
df_uni.plot(ax=ax)
plt.title("Original Fuel Price")
plt.xlabel("")
plt.ylabel("AUD$")
plt.show()

adfuller(df_uni)

Without differencing, $p$ value is .0006 and the test statistic is -4.235. Thus, null hypothesis that the time series is non-stationary can be rejected.

In [ ]:
# Take first difference and test again
df_uni_1diff = df_uni.diff(1).dropna()

fig, ax = plt.subplots(figsize=(8,5))
df_uni_1diff.plot(ax=ax)
plt.title("First Differencing of Fuel Price")
plt.xlabel("")
plt.ylabel("First differenced AUD$")
plt.show()

adfuller(df_uni_1diff)

Taking first differencing makes the time series more stationary.

In [ ]:
# Try log-return and test again
df_uni_logreturn = np.log(df_uni/df_uni.shift(1)).dropna()

fig, ax = plt.subplots(figsize=(8,5))
df_uni_logreturn.plot(ax=ax)
plt.title("Log-Return of Fuel Price")
plt.xlabel("")
plt.ylabel("Log-returned AUD$")
plt.show()

adfuller(df_uni_logreturn)

Log-return produces similar result. However, I will stick to the original time-series for the following analysis and forecast.

In [ ]:
# Train test split
train = df_uni.iloc[:round(df_uni.shape[0]*.75)]
test = df_uni.iloc[round(df_uni.shape[0]*.75):]

fig, ax = plt.subplots(figsize=(10,5))
train.plot(label='train', ax=ax)
test.plot(label='test', ax=ax)
plt.title("Fuel Price in Training and Testing Sets")
plt.xlabel("")
plt.ylabel("AUD$")
plt.legend()
plt.show()

## Modelling<a id='model'></a>

In [ ]:
# plot ACF and PACF graphs
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,8))

plot_acf(train, lags=12, zero=False, ax=ax1)
plot_pacf(train, lags=12, zero=False, ax=ax2)
plt.show()

&emsp;The ACF and PACF plots both did do not show signs of tailing off. The determination of orders is inconclusive.

In [ ]:
# Search for the best order by AIC and BIC
order_aic_bic = []

for p in range(5):
    for q in range(5):
        try:
            model = ARIMA(train, order=(p,0,q))
            results = model.fit()
            order_aic_bic.append((p, q, results.aic, results.bic))
        except:
            order_aic_bic.append((p, q, None, None))

order_df = pd.DataFrame(order_aic_bic, columns=['p', 'q', 'aic', 'bic'])

In [ ]:
order_df.sort_values(['aic', 'bic'])

&emsp;The order (0,0,2) yields the best AIC value.

In [ ]:
# Fit model
model = ARIMA(train, order=(0,0,2))
results = model.fit()
# Summary of fit model
print(results.summary())
# Line plot of residuals
residuals = results.resid
residuals.plot()
plt.title("Residual Plot")
plt.xlabel("")
plt.show()
# Mean absolute error (MAE)
mae = np.mean(np.abs(residuals))
print(f'MAE:{mae}')
# Density plot of residuals - white Gaussian noise should be centred around 0
results.plot_diagnostics(figsize=(12,12))
plt.show()
# Summary stats of residuals
print(residuals.describe())

&emsp;The Prob(Q) and Prob(JB) are both non-significant. So the model's residuals are not correlated and of normal distribution. The points on the Normal Q-Q plot do not lie along the red line. The model needs to improve.

In [ ]:
# Forecast (one-step ahead in-sample prediction)
forecast = results.get_prediction(start=-15)
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure()
plt.plot(train.index.to_series().astype('str'), train, label='observed')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str'), mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str'), conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis tick labels
plt.xticks(rotation=90)
plt.legend()
plt.title("One-Step Ahead In-Sample Prediction")
plt.ylabel("AUD$")
plt.show()

In [ ]:
# Forecast (Dynamic in-sample prediction)
forecast = results.get_prediction(start=-10, dynamic=True) # start=-25
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure()
plt.plot(train.index.to_series().astype('str'), train, label='observed')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str'), mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str'), conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis tick labels
plt.xticks(rotation=90)
plt.legend()
plt.title("Dynamic In-Sample Prediction")
plt.ylabel("AUD$")
plt.show()

In [ ]:
# Forecast (Out-sample prediction)
forecast = results.get_forecast(steps=10)
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure()
plt.plot(train.index.to_series().astype('str'), train, label='train')
plt.plot(test.index.to_series().astype('str'), test, label='test')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str'), mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str'), conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90)
plt.legend()
plt.title("Model Validation")
plt.ylabel("AUD$")
plt.show()

&emsp;Only the one step ahead in-sample prediction is able to predict relatively accurately than dynamic in-sample and forecast. Without the information from the previous data point, the model failed to make useful predictions. In the following, I try to put seasonal pattern consideration into the model to account for the repeatedly spikes.

### Seasonal Pattern

In [ ]:
# De-trend the time-series to determine the cycle period
df_detrend = train - train.rolling(5).mean()
df_detrend.dropna()
df_detrend.plot()
plt.title("Detrended Time Series")
plt.xlabel("")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,4))

plot_acf(df_detrend.dropna(), lags=22, zero=False, ax=ax) # lags=67
plt.show()

In [ ]:
decomp_results = seasonal_decompose(df_uni.values, period=3) # period=4
decomp_results.plot()
plt.show()

In [ ]:
# Seasonal differencing 
df_uni_season_diff = train.diff(3).dropna() # .diff(4)
df_uni_season_diff.plot()
plt.title("Seasonal Differencing of Training Set")
plt.xlabel("")
plt.show()

In [ ]:
# plot seasonal ACF and PACF graphs
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,8))
lags = [i for i in range(3,13,3)]
plot_acf(df_uni_season_diff, lags=lags, ax=ax1)
plot_pacf(df_uni_season_diff, lags=lags, ax=ax2)
plt.show()

&emsp;The ACF and PACF again do not show any tailing off. The orders are still inconclusive.

In [ ]:
# Apply SARIMA to capture seasonal patterns
model = SARIMAX(train, order=(0,0,2), seasonal_order=(0,1,0,3))
results = model.fit()

# summary of fit model
print(results.summary())
# line plot of residuals
residuals = results.resid
residuals.plot()
plt.title("Residual Plot")
plt.xlabel("")
plt.show()
# Mean absolute error (MAE)
mae = np.mean(np.abs(residuals))
print(f'MAE:{mae}')
# density plot of residuals - white Gaussian noise should be centred around 0
results.plot_diagnostics(figsize=(12,12))
plt.show()
# residuals.plot(kind='kde')
# plt.show()
# summary stats of residuals
print(residuals.describe())

&emsp;pmdarima estimator is used to find the best orders with given orders of differencing for both non-seasonal and seasonal patterns with a period of 3.

In [ ]:
# Searching over model orders
results = pm.auto_arima(train.values, seasonal=True, d=0, D=1, m=3, information_criterion='aic', trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)
# Update parameters with new observations
# results.update(df_new)
print(results.summary())
results.plot_diagnostics(figsize=(10,8))
plt.show()

print(f'MAE: {np.mean(np.abs(results.resid()))}')

joblib.dump(results, "model.pkl")

# Load model
# model_results_object = joblib.load("model.pkl")

&emsp;The mean absolute error reduced and the data points after 0 in correlogram are all insignificant. Additionally, the histogram has a better distribution that resembles the normal one.

In [ ]:
results

In [ ]:
# Apply SARIMA to capture seasonal patterns
model = SARIMAX(train, order=(2,0,0), seasonal_order=(0,1,0,3))
results = model.fit()

In [ ]:
# Forecast (one-step ahead in-sample prediction)
forecast = results.get_prediction(start=-10)
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(train.index.to_series().astype('str'), train, label='observed')
# plt.plot(train.index.to_series().astype('str'), train, label='validate')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str'), mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str'), conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90)
plt.legend()
plt.title("One-Step Ahead In-Sample Prediction")
plt.xlabel("")
plt.ylabel("AUD$")
plt.show()

In [ ]:
# Forecast (Dynamic in-sample prediction)
forecast = results.get_prediction(start=-10, dynamic=True)
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(train.index.to_series().astype('str'), train, label='observed')
# plt.plot(test.index.to_series().astype('str'), test, label='validate')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str'), mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str'), conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90)
plt.legend()
plt.title("Dynamic In-Sample Prediction")
plt.xlabel("")
plt.ylabel("AUD$")
plt.show()

In [ ]:
# Forecast (Out-sample prediction)
forecast = results.get_forecast(steps=20)
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(df_uni.index.to_series().astype('str'), df_uni, label='observed')
plt.plot(test.index.to_series().astype('str'), test, label='validate')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str'), mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str'), conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90, fontsize=6)
plt.legend()
plt.title("Model Validation")
plt.xlabel("")
plt.ylabel("AUD$")
plt.show()

&emsp;The seasonal model is unable to predict accurately the overall trend. Next, an exogenous feature is put into the training model.

## Exogenous Feature<a id='exog'></a>

In [ ]:
# Connect database for weather data
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password"
)
mycursor = mydb.cursor()

# Extract dataset
mycursor.execute(
    "SELECT CONCAT(year, '-', month) AS month, ROUND(AVG(temp_max), 1) AS temp_max\
    FROM base.weather\
    WHERE year >= 2022\
    GROUP BY year, month"
)

result = mycursor.fetchall()

for x in result:
    print(x)

In [ ]:
df_temp = pd.DataFrame(result, columns=['date','temp'])
df_temp.head()

In [ ]:
# Concat both features
df = pd.concat([pd.DataFrame(df_uni).reset_index(), df_temp['temp']], axis=1)
df['date'] = df['date'].astype(str)
df = df.set_index('date')
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

sns.lineplot(df, x='date', y='price', c='tab:blue', label='fuel', ax=ax, legend=False)

ax2 = ax.twinx()
sns.lineplot(df, x='date', y='temp', c='tab:orange', label='temp', ax=ax2, legend=False)

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

ax.figure.legend()
plt.title("Fuel Price and Temperature from 2022 to 2024")
plt.xlabel("")
plt.show()

&emsp;Although temperature in Perth has a clear seasonal pattern, it does not have a clear relationship with the fuel price.

In [ ]:
# Train test split
train = df.iloc[:round(df.shape[0]*.75)]
test = df.iloc[round(df.shape[0]*.75):]

fig, ax = plt.subplots(figsize=(8,5))
sns.lineplot(train, x='date', y='price', c='tab:blue', label='train', legend=False, ax=ax)
sns.lineplot(test, x='date', y='price', c='tab:orange', label='test', legend=False, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_title("Training and Testing Sets of Fuel Price")
ax.set_xlabel("")
ax.legend()
plt.show()

# Apply SARIMA to capture seasonal patterns
model = SARIMAX(train['price'], order=(2,0,0), seasonal_order=(0,1,0,3), exog=train['temp'])
results = model.fit()
# summary of fit model
print(results.summary())
# line plot of residuals
residuals = results.resid
residuals.plot()
plt.title("Residual Plot")
plt.xlabel("")
plt.show()
# Mean absolute error (MAE)
mae = np.mean(np.abs(residuals))
print(f'MAE:{mae}')
# density plot of residuals - white Gaussian noise should be centred around 0
results.plot_diagnostics(figsize=(12,12))
plt.show()
# residuals.plot(kind='kde')
# plt.show()
# summary stats of residuals
print(residuals.describe())

In [ ]:
# Forecast (one-step ahead in-sample prediction)
forecast = results.get_prediction(start=-10, exog=train['temp'])
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(train.index, train['price'], label='observed')
# plt.plot(train.index.to_series().astype('str'), train, label='validate')
# Prediction
plt.plot(train.index[-10:], mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(train.index[-10:], conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90)
plt.legend()
plt.title("One-Step Ahead In-Sample Prediction")
plt.ylabel("AUD$")
plt.show()

In [ ]:
# Forecast (Dynamic in-sample prediction)
forecast = results.get_prediction(start=-10, dynamic=True, exog=train['temp'])
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(train.index, train['price'], label='observed')
# plt.plot(test.index.to_series().astype('str'), test, label='validate')
# Prediction
plt.plot(train.index[-10:], mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(train.index[-10:], conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90)
plt.legend()
plt.title("Dynamic In-Sample Prediction")
plt.ylabel("AUD$")
plt.show()

In [ ]:
# Forecast (Out-sample prediction)
forecast = results.get_forecast(steps=test.shape[0], exog=test['temp'])
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(train.index, train['price'], label='observed')
plt.plot(test.index, test['price'], label='validate')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str').str[:-3], mean_forecast.values, color='red', label='forecast')
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str').str[:-3], conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90, fontsize=6)
plt.legend()
plt.title("Model Validation")
plt.ylabel("AUD$")
plt.show()

&emsp;Overall, the model with additional exogenous feature produces a little bit more information to the prediction. It lightly captured the downward trend in the validation set.

In [ ]:
# Final forecast for future price (Out-sample prediction)
# Apply SARIMA to capture seasonal patterns
model = SARIMAX(df['price'], order=(2,0,0), seasonal_order=(0,1,0,3), exog=df['temp'])
results = model.fit()

forecast = results.get_forecast(steps=24, exog=df['temp'].iloc[-24:])
# Mean forecast
mean_forecast = forecast.predicted_mean
# Confident intervals
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10,5))
plt.plot(df.index, df['price'], label='observed')
# Prediction
plt.plot(mean_forecast.index.to_series().astype('str').str[:-3], mean_forecast.values, color='red', label='forecast')
plt.plot([df.index[0], mean_forecast.index.to_series().astype('str').str[:-3][-1]], [np.average(mean_forecast.values)]*2)
# Uncertainty area
plt.fill_between(mean_forecast.index.to_series().astype('str').str[:-3], conf_int['lower price'], conf_int['upper price'], color='pink')
# X-axis labels
plt.xticks(rotation=90, fontsize=6)
plt.legend()
plt.title("2-Year Forecast on Fuel Price")
plt.ylabel("AUD$")
plt.show()

&emsp;According to the forecast, the average fuel price in Perth metro area will fluctuate around `AUD$177/100L` in the coming two years. The forecast does not reproduce the spikes as happened in the past. To improve, more data from the past is recommended.